<a href="https://colab.research.google.com/github/markaaronslater/NMT/blob/master/playground.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%load_ext autoreload
%autoreload 2

from google.colab import drive
drive.mount('/content/gdrive')

!pip install subword-nmt # for segmenting words into subwords
!pip install stanza # for tokenizing corpus and tagging with morphological data
!pip install sacremoses # for detokenizing model predictions
!pip install sacrebleu # for evaluation

# make sure using GPU
# (Runtime -> Change runtime type -> Hardware accelerator = GPU).
!nvidia-smi

%cd /content/gdrive/My Drive/NMT

from src.model_utils import load_pretrained
from src.predict import predict
from src.evaluate import evaluate

import stanza
from subword_nmt.apply_bpe import BPE
# recommended path to project root directory: place cloned NMT folder in 'My Drive' folder of Google Drive account:
path = '/content/gdrive/My Drive/NMT/'
model_name = 'old_replica/' # name of pre-trained model to load
checkpoint_path = path + 'checkpoints/' + model_name
corpus_path = path + 'corpuses/iwslt16_en_de/subword_segmented/'
### ??how to give them access to my pre-trained model?? too large to commit to github??

model, test_batches, src_word_to_idx, idx_to_trg_word = load_pretrained(checkpoint_path=checkpoint_path, corpus_path=corpus_path)
stanza.download(lang='de', processors='tokenize,mwt,pos')
stanza_de_processor = stanza.Pipeline(lang='de', processors='tokenize,mwt,pos', tokenize_no_ssplit=True)
bpe = BPE(corpus_path + 'bpe_codes', set(src_word_to_idx))





The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
     |████████████████████████████████| 235kB 2.8MB/s 
     |████████████████████████████████| 890kB 2.6MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=bba08adb6ab912c8c9622af3db3a4e6d86bad59a0aaf136093e182c28287048a
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [ ]:
# a) translation playground:
# place any number of whatever German sentences you want as strings inside following list:
input = ["Dies ist ein deutscher Testsatz. Wird das Modell es erfolgreich übersetzen können?", 
         "Wenn nicht, wird diese Demo nicht sehr beeindruckend sein ...",
         "Ich empfehle, einen englischen Satz zu erstellen und ihn dann mit Google Translate in Deutsch umzuwandeln."]

# determined via Google Translate:
sample_targets = ["This is a German test sentence. Will the model be able to translate it successfully?",
                  "If it doesn't, then this demo will not be very impressive...",
                  "I recommend coming up with an English sentence, and then using Google Translate to convert it to German."]

translations = demo_translate(input, stanza_de_processor, translator, src_word_to_idx, idx_to_trg_word, bpe, device='cuda:0', bsz=10)
for translation in translations:
    print(translation)
print()


In [ ]:
# if targets are available, evaluate:
bleu = evaluate(dev_translations, dev_references)

In [ ]:
# b) replicate BLEU score on test set



# download iwslt17 test set
